# 放弃了使用新浪博客,刚好学习一下notebook的使用方法

测试

![Images](https://raw.githubusercontent.com/KannShi/Udacity_DL/master/img/regularization.png)

In [ ]:
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(None, image_size * image_size))